In [1]:
import pandas as pd
import requests
import json
import numpy as np

In [2]:
# Read in the FARA data as a Pandas DataFrame
filepath = """../input-data/2019 Food Access Research Atlas Data/Food Access Research Atlas.csv"""
data = pd.read_csv(filepath)

In [3]:
# Limit the FARA data to the state of Texas
texas = data.loc[data["State"]=="Texas",:]

# Select only the relevant columns
texas = texas[['CensusTract','State','County','Urban','Pop2010','OHU2010','LILATracts_1And10','LILATracts_halfAnd10','HUNVFlag','LowIncomeTracts',
'PovertyRate','MedianFamilyIncome','LA1and10','LAhalfand10','LATracts_half','LALOWI1_10','LALOWI05_10','lahunvhalf','lahunvhalfshare']]

In [4]:
# Open McDonald's JSON file
json_file = open("../input-data/mcdata.json")
mcdonalds = json.load(json_file)["features"]

# Store all the McDonalds in Texas
mcdonalds_tx = [store for store in mcdonalds if store['properties']['state']=='TX']

# Store lats and lons of each McDonald's restaurant in US
lats = [store['geometry']['coordinates'][1] for store in mcdonalds_tx]
lons = [store['geometry']['coordinates'][0] for store in mcdonalds_tx]

In [5]:
# Store the base URL - this API takes geographical coordinates and returns the Census Tract that those coordinates fall into
base_url = "https://geo.fcc.gov/api/census/area?"

# Initialize an empty list to store the census tract info for each store
census_tracts = []

# Loop through the stores in texas using enumerate
for i in range(len(mcdonalds_tx)):

    # create a url for each store using the latitude and longitutde and base
    store_url = f'{base_url}lat={lats[i]}&lon={lons[i]}&censusYear=2010&format=json'

   # Run API request for each store
    try:
        # Store the results
       results = requests.get(store_url).json()
       
       # Store the tract number - NOTE: only the first 11 digits because the API number is more specific than the Food Access Research Atlas
       tract = results["results"][0]["block_fips"][0:-4]

        # Append the tract number to a list
       census_tracts.append(tract)

    except:
       print("No tract available")
       pass


In [6]:
# Convert the census tract numbers to integers
census_tracts = [int(tract) for tract in census_tracts]

# Store the census tracts with a McDonald's as a dataframe
mctracts = pd.DataFrame(census_tracts, columns=["CensusTract"])

# Add a column to flag the census tracts
mctracts["HasMcD"] = 1

# Group by the tract number
mctracts_grp = mctracts.groupby("CensusTract")

# Using the grouped dataframe, create a column summing the flag column to show the number of McDonalds per census tract
mctracts_num = pd.DataFrame(mctracts.groupby("CensusTract")["HasMcD"].sum())

# Drop the duplicates in the original dataframe
mctracts = mctracts.drop_duplicates("CensusTract")

# Merge the original dataframe to the one with a count column, in order to have both a flag and a count column
mctracts_final = pd.merge(mctracts,mctracts_num,on="CensusTract")

# Rename the columns
mctracts_final = mctracts_final.rename(columns = {"HasMcD_x":"HasMcD","HasMcD_y":"CountMcD"})


In [7]:
# Merge the McDonald's count and flag columns onto the original FARA data
tx_tracts = texas.merge(mctracts_final,how="left", on="CensusTract")

# Replace the null values with zeros that represent no McDonald's in that census tract
tx_tracts[["HasMcD","CountMcD"]] = tx_tracts[["HasMcD","CountMcD"]].fillna(0)


In [21]:
# Save the pandas dataframe to a sqlite file using sqlalchemy
from sqlalchemy import create_engine
import sqlite3
# Create an engine connection to the sqlite database
conn = sqlite3.connect('../database/outputdata.sqlite')



In [22]:
# Covert the dataframe to sql
tx_tracts.to_sql('dataframe', conn, if_exists='replace', index=False)

5238